In [1]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import glob
from nltk.stem.porter import PorterStemmer

In [2]:
def readFile(directory):
    listFile = glob.glob(directory+"/*.txt")
    return listFile

In [3]:
def readPerWord(file):
    words = []
    f = open(file, 'r') #open file
    for line in f:
        words.extend(line.split(' ')) #tiap satu baris split per kata
        for word in words: #regex
            re.sub(r'\W$','',word)

    f.close()

    return words

In [4]:
def removeEnter(listOfWords): #['<BODY>\n']
    for i in range(len(listOfWords)):
        listOfWords[i] = listOfWords[i].replace('\n','')
        
    return listOfWords

In [5]:
def removeNull(listOfWords): #['','','eat','food','','']
    listOfWords = list(filter(None, listOfWords))
    
    return listOfWords

In [6]:
def caseFolding(listOfWords):
    for i in range(len(listOfWords)):
        listOfWords[i] = listOfWords[i].casefold()
        
    return listOfWords

<h3>Load Stop Word from NLTK</h3>

In [7]:
stop_words = stopwords.words('english')
stop_words.append("you'r")

In [8]:
def removeStopWord(listOfWords):
    for i in listOfWords:
        if i in stop_words:
            listOfWords.remove(i)
        
    return listOfWords

<h3>Load Lemmatizer from NLTK</h3>

In [9]:
lemmatizer = WordNetLemmatizer()

In [10]:
def lemmatization(listOfWords):
    for i in range(len(listOfWords)):
        listOfWords[i] = lemmatizer.lemmatize(listOfWords[i])
        
    return listOfWords

<h3>Load Stemming from NLTK</h3>

In [11]:
def stemming(listOfWords):
    porter = PorterStemmer()
    stemmed = [porter.stem(word) for word in listOfWords]
    
    return stemmed

<h1>Test</h1>

In [19]:
listOfFiles = readFile('DataRouter')
listOfWordsOfFile = []
token_sesudah= []

for i in listOfFiles: #iterasi tiap file
    listOfWords = readPerWord(i)
    
    #remove tags
    re.sub(r'<DATE>|</DATE>|<TITLE>|</TITLE>|<BODY>|</BODY>',listOfWords,'')
    
    listOfWords = removeEnter(listOfWords)
    listOfWords = removeNull(listOfWords)
    listOfWords = caseFolding(listOfWords)
    listOfWords = removeStopWord(listOfWords)
    listOfWords = lemmatization(listOfWords)
#     listOfWords = stemming(listOfWords)

    listOfWordsOfFile.append(listOfWords)
    
    for i in listOfWords:
        token_sesudah.append(i)
        
# listOfWords = removeEnter(listOfWords)

TypeError: unhashable type: 'list'

In [13]:
# for i in token_sesudah:
#     print(i)
#     print('--')

# Token

In [14]:
def make_token(list_dokumen):
    list_hasil = []
    
    for i in list_dokumen:
        token = readPerWord(i)
        for j in token:
            list_hasil.append(j)
            
    return list_hasil

# Term

In [15]:
def make_term(list_token):
    list_term = []
    for i in list_token:
        if i not in list_term:
            list_term.append(i)
    return list_term


In [16]:
def concate_term_newline(list_in):
    hasil = ''
    for i in list_in:
        hasil = hasil + "\n " + i
        
    return hasil

# Tulis Hasil

In [17]:
token_sebelum = make_token(listOfFiles)
term_sebelum = make_term(token_sebelum)
term_sesudah = make_term(token_sesudah)

token_sebelum_str = concate_term_newline(token_sebelum)
term_sebelum_str = concate_term_newline(term_sebelum)
token_sesudah_str = concate_term_newline(token_sesudah)
term_sesudah_str = concate_term_newline(term_sesudah)

In [18]:
saveFileToken = open("token_sebelum_pemprosesan.txt", 'w' )
saveFileToken.write(token_sebelum_str)
saveFileToken.close()

saveFileTerm = open("term_sebelum_pemprosesan.txt", 'w' )
saveFileTerm.write(term_sebelum_str)
saveFileTerm.close()

saveFileTokenS = open("token_sesudah_pemprosesan.txt", 'w' )
saveFileTokenS.write(token_sesudah_str)
saveFileTokenS.close()

saveFileTerms = open("term_sesudah_pemprosesan.txt", 'w' )
saveFileTerms.write(term_sesudah_str)
saveFileTerms.close()